Recurrent Neural networks
=====

### RNN  

<img src ="imgs/rnn.png" width="20%">

A recurrent neural network (RNN) is a class of artificial neural network where connections between units form a directed cycle. This creates an internal state of the network which allows it to exhibit dynamic temporal behavior.

In [1]:
import keras
output_dim = 100

Using TensorFlow backend.


In [2]:
keras.layers.recurrent.SimpleRNN(output_dim, 
                                 init='glorot_uniform', inner_init='orthogonal', activation='tanh', 
                                 W_regularizer=None, U_regularizer=None, b_regularizer=None, 
                                 dropout_W=0.0, dropout_U=0.0)

#### Backprop Through time  

Contrary to feed-forward neural networks, the RNN is characterized by the ability of encoding longer past information, thus very suitable for sequential models. The BPTT extends the ordinary BP algorithm to suit the recurrent neural
architecture.

<img src ="imgs/rnn2.png" width="45%">

In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T
import keras 
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing import image
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import imdb
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from keras.layers.core import Activation, TimeDistributedDense, RepeatVector
from keras.callbacks import EarlyStopping, ModelCheckpoint

import tensorflow as tf

tf.python.control_flow_ops = tf

#### IMDB sentiment classification task

This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. 

IMDB provided a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. 

There is additional unlabeled data for use as well. Raw text and already processed bag of words formats are provided. 

http://ai.stanford.edu/~amaas/data/sentiment/

#### Data Preparation - IMDB

In [5]:
max_features = 20000
maxlen = 100  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print("Loading data...")
# (X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features, test_split=0.2)
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Example:')
print(X_train[:1])

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

max_epoch = 50

Loading data...
25000 train sequences
25000 test sequences
Example:
[ [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 1

In [6]:
X_train.shape

(25000, 100)

#### Model building 

In [7]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SimpleRNN(128))  
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(X_test, y_test))
acc , loss = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
24992/25000 [============================>.] - ETA: 0s
Test accuracy: 0.583960152206
Test loss: 0.68604


### LSTM  

A LSTM network is an artificial neural network that contains LSTM blocks instead of, or in addition to, regular network units. A LSTM block may be described as a "smart" network unit that can remember a value for an arbitrary length of time. 

Unlike traditional RNNs, an Long short-term memory network is well-suited to learn from experience to classify, process and predict time series when there are very long time lags of unknown size between important events.

<img src ="imgs/gru.png" width="60%">

In [8]:
keras.layers.recurrent.LSTM(output_dim, init='glorot_uniform', inner_init='orthogonal', 
                            forget_bias_init='one', activation='tanh', 
                            inner_activation='hard_sigmoid', 
                            W_regularizer=None, U_regularizer=None, b_regularizer=None, 
                            dropout_W=0.0, dropout_U=0.0)

### GRU  

Gated recurrent units are a gating mechanism in recurrent neural networks. 

Much similar to the LSTMs, they have fewer parameters than LSTM, as they lack an output gate.

In [9]:
keras.layers.recurrent.GRU(output_dim, init='glorot_uniform', inner_init='orthogonal', 
                           activation='tanh', inner_activation='hard_sigmoid', 
                           W_regularizer=None, U_regularizer=None, b_regularizer=None, 
                           dropout_W=0.0, dropout_U=0.0)

### Your Turn! - Hands on Rnn

In [10]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))

# Play with those! try and get better results!
model.add(SimpleRNN(128))  
#model.add(GRU(128))  
#model.add(LSTM(128))  

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          nb_epoch=max_epoch, validation_data=(X_test, y_test))
acc , loss = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/50
25000/25000 [==============================] - 60s - loss: 0.7131 - acc: 0.5300 - val_loss: 0.6646 - val_acc: 0.5749
Epoch 2/50
25000/25000 [==============================] - 61s - loss: 0.5396 - acc: 0.7244 - val_loss: 1.0878 - val_acc: 0.6199
Epoch 3/50
25000/25000 [==============================] - 60s - loss: 0.4727 - acc: 0.7913 - val_loss: 0.4992 - val_acc: 0.7603
Epoch 4/50
25000/25000 [==============================] - 60s - loss: 0.4305 - acc: 0.8233 - val_loss: 0.4692 - val_acc: 0.7978
Epoch 5/50
25000/25000 [==============================] - 60s - loss: 0.3811 - acc: 0.8468 - val_loss: 0.4556 - val_acc: 0.7897
Epoch 6/50
25000/25000 [==============================] - 60s - loss: 0.3603 - acc: 0.8505 - val_loss: 0.4583 - val_acc: 0.8065
Epoch 7/50
25000/25000 [==============================] - 101s - loss: 0.4698 - acc: 0.7655 - val_loss: 0.6272 - val_acc: 0.6393
Epoch 8/50
25000/25000 [=====

In [11]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))

# Play with those! try and get better results!
# model.add(SimpleRNN(128))  
model.add(GRU(128))  
# model.add(LSTM(128))  

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          nb_epoch=max_epoch, validation_data=(X_test, y_test))
acc , loss = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/50
25000/25000 [==============================] - 125s - loss: 0.4269 - acc: 0.7995 - val_loss: 0.3627 - val_acc: 0.8408
Epoch 2/50
25000/25000 [==============================] - 125s - loss: 0.2247 - acc: 0.9136 - val_loss: 0.3649 - val_acc: 0.8539
Epoch 3/50
25000/25000 [==============================] - 125s - loss: 0.1128 - acc: 0.9606 - val_loss: 0.4526 - val_acc: 0.8387
Epoch 4/50
25000/25000 [==============================] - 126s - loss: 0.0564 - acc: 0.9818 - val_loss: 0.5723 - val_acc: 0.8390
Epoch 5/50
25000/25000 [==============================] - 125s - loss: 0.0290 - acc: 0.9908 - val_loss: 0.6926 - val_acc: 0.8381
Epoch 6/50
25000/25000 [==============================] - 128s - loss: 0.0199 - acc: 0.9936 - val_loss: 0.7457 - val_acc: 0.8370
Epoch 7/50
25000/25000 [==============================] - 128s - loss: 0.0187 - acc: 0.9940 - val_loss: 0.7592 - val_acc: 0.8323
Epoch 8/50
25000/25000 

In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))

# Play with those! try and get better results!
# model.add(SimpleRNN(128))  
# model.add(GRU(128))  
model.add(LSTM(128))  

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          nb_epoch=max_epoch, validation_data=(X_test, y_test))
acc , loss = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/50
25000/25000 [==============================] - 137s - loss: 0.4363 - acc: 0.7973 - val_loss: 0.3679 - val_acc: 0.8391
Epoch 2/50
25000/25000 [==============================] - 136s - loss: 0.2392 - acc: 0.9094 - val_loss: 0.4281 - val_acc: 0.8330
Epoch 3/50
25000/25000 [==============================] - 140s - loss: 0.1404 - acc: 0.9487 - val_loss: 0.5021 - val_acc: 0.8389
Epoch 4/50
25000/25000 [==============================] - 139s - loss: 0.0782 - acc: 0.9743 - val_loss: 0.5590 - val_acc: 0.8310
Epoch 5/50
25000/25000 [==============================] - 138s - loss: 0.0551 - acc: 0.9814 - val_loss: 0.7147 - val_acc: 0.8321
Epoch 6/50
25000/25000 [==============================] - 137s - loss: 0.0370 - acc: 0.9881 - val_loss: 0.8151 - val_acc: 0.8163
Epoch 7/50
25000/25000 [==============================] - 146s - loss: 0.0253 - acc: 0.9922 - val_loss: 0.7970 - val_acc: 0.8276
Epoch 8/50
25000/25000 

# Sentence Generation using RNN(LSTM)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
200285/200285 [==============================] - 314s - loss: 1.9922   

----- diversity: 0.2
----- Generating with seed: "n short, more pleasantly--one is much mo"
n short, more pleasantly--one is much most the stread the conticion of the contime the condeption of the depreted of the deperions and which it is called the most the self the strean the most the continity and the word to be a cause of the conticion of the contime the aster the stare of the conticion of the conticion of the contime the love the word the deperient the some and into the same the hand and a propers and a stread and a stare

--------------------------------------------------
Iteration 2
Epoch 1/1
200285/200285 [==============================] - 310s - loss: 1.6420   

----- diversity: 0.2
----- Generating with seed: "le certainty with which his inst

/home/ubuntu/anaconda3/envs/deep-learning/lib/python3.5/site-packages/ipykernel/__main__.py:52: RuntimeWarning: divide by zero encountered in log


he world, and prese

--------------------------------------------------
Iteration 12
Epoch 1/1
200285/200285 [==============================] - 312s - loss: 1.3828   

----- diversity: 0.2
----- Generating with seed: "bout the good, true, and beautiful,
and "
bout the good, true, and beautiful,
and in the same time as the soul of the soul and in the sense of the sense of the most and stronger and constitute that the sense of the sentiment of the soul as the same out of the soul as a something that it is the self--it is also the soul as the soul and the soul as its souls and interpretation, and the same with the sense of the sense of which the sense of the sense of the sense of the soul and a

--------------------------------------------------
Iteration 13
Epoch 1/1
200285/200285 [==============================] - 313s - loss: 1.3764   

----- diversity: 0.2
----- Generating with seed: "le doses is very great,
nevertheless; th"
le doses is very great,
nevertheless; there is all the sens